# Introduction and Motivation

In the era of digital media, music streaming platforms have transformed how we discover and enjoy music. Services like Spotify have millions of tracks available at our fingertips, making music recommendation systems essential for enhancing user experience. Recognizing the significance of these systems, our team chose to explore the “Spotify Tracks, Genre, Audio Features” dataset from Kaggle (https://www.kaggle.com/datasets/pepepython/spotify-huge-database-daily-charts-over-3-years/data) using PySpark to analyze trends and build a music recommendation model.

Our motivation for selecting this dataset is rooted in a desire to engage with a more intriguing and relatable subject matter than the typical technical datasets we’ve encountered in previous classes. Music is a universal language, and understanding how recommendation systems work within this context is both exciting and relevant. Music recommendation models are increasingly important as the amount of data generated online continues to grow, impacting how users interact with streaming services.

The primary objectives of our project are twofold:
1. Data Analysis: We aim to understand trends within the music data by examining how different genres relate to specific audio feature metrics. For example, we want to investigate whether certain genres consistently exhibit higher danceability scores or how energy levels vary across genres.
2. Model Building: Utilizing machine learning techniques, we plan to develop a music recommendation model. This model will leverage the insights gained from our data analysis to suggest tracks that align with user preferences based on audio features.

Our project focuses on mainstream music, as the dataset predominantly features popular tracks. While we have considered algorithms like K-Nearest Neighbors for recommendation, we will determine the most suitable machine learning methods as we progress, ensuring they align with our dataset characteristics and project goals.

In [139]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("Music Database Analysis").getOrCreate()

# Path to your CSV file
csv_path = "../Music_Database.csv"

# Load the CSV file into a Spark DataFrame
music_df = spark.read.csv(csv_path, header=True, inferSchema=True)

# Show the first few rows to verify
music_df.show()

+-----------+--------------------+------------------+---------------+--------------------+------------+-----------------+----------------+--------+--------------------+------------+------------+---------------+------------------+------------------+---+--------+----+-----------+-------------------+----------------+----------+-------+-------+-----------+--------------+---------+------------------+-------------------+--------------+-------------+-------+-----------+------+------+--------+---------+----------------+----+----+-------+-----+-----+----+-----+-----+-----+---+---+--------+---+------+---------+----+----+------------+---------+----------+--------+-------+----+-----+----+-----+------------+-------+----+----+-------+--------+-----+--------+--------+-------+----------+-------------------+------------+-------------------+-------------------+------------------+-------------+-------------------+------------------+-------------------+-----------+------------------+-------------+--------

24/11/10 17:34:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

In [140]:
# Print the schema of the DataFrame
music_df.printSchema()

root
 |-- Country0: string (nullable = true)
 |-- Uri: string (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Album/Single: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Artist_followers: string (nullable = true)
 |-- Explicit: string (nullable = true)
 |-- Album9: string (nullable = true)
 |-- Release_date: string (nullable = true)
 |-- Track_number: string (nullable = true)
 |-- Tracks_in_album: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acoustics: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveliness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- duration_ms: 

In [141]:
from pyspark.sql.functions import col, sum

# Calculate the number of nulls in each column
null_counts = music_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in music_df.columns])
null_counts.show()

24/11/10 17:34:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

+--------+---+----------+-----+------+------------+-----+----------------+--------+------+------------+------------+---------------+------------+------+---+--------+----+-----------+---------+----------------+----------+-------+-----+-----------+--------------+---------+------------------+-------------------+--------------+-------------+-------+-----------+------+------+--------+---------+----------------+----+----+-------+-----+-----+----+-----+-----+-----+---+---+--------+---+------+---------+----+----+------------+---------+----------+--------+-------+-----+-----+-----+-----+------------+-------+-----+-----+-------+--------+-----+--------+--------+-------+----------+-----------------+------------+---------+--------+------------+-------------+----------+-------------+-------------+-----------+------------------+-------------+----------+---------+-------------+--------------+-----------+--------------+--------------+---------------+---------------+-----+--------------+-------------+--

In [142]:
# List of columns to include in the statistical summary
numerical_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                     'acoustics', 'instrumentalness', 'liveliness', 'valence', 'tempo',
                     'duration_ms', 'time_signature', 'Artist_followers', 'Popularity']

# Since these columns are currently strings, we'll convert them to numeric types in the next step
from pyspark.sql.functions import col

# Convert columns to DoubleType
for column in numerical_columns:
    music_df = music_df.withColumn(column, col(column).cast("double"))

music_df.printSchema()

root
 |-- Country0: string (nullable = true)
 |-- Uri: string (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Album/Single: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Artist_followers: double (nullable = true)
 |-- Explicit: string (nullable = true)
 |-- Album9: string (nullable = true)
 |-- Release_date: string (nullable = true)
 |-- Track_number: string (nullable = true)
 |-- Tracks_in_album: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acoustics: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveliness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: 

In [143]:
# Count before removing duplicates
total_records = music_df.count()

# Remove duplicates based on 'Uri'
music_df = music_df.dropDuplicates(['Uri'])

# Count after removing duplicates
cleaned_records = music_df.count()
print(f"Duplicates removed: {total_records - cleaned_records}")
print(f"Records after duplicate filter: {cleaned_records}")

Duplicates removed: 108213
Records after duplicate filter: 62420


In [144]:
from pyspark.sql.functions import col

# Define popularity threshold
popularity_threshold = 50

# Filter songs with Popularity >= 50
filtered_df = music_df.filter(col('Popularity') >= popularity_threshold)
print(f"Records after popularity filter: {filtered_df.count()}")

Records after popularity filter: 47972


In [145]:
cleaned_df = filtered_df.na.drop()
print(f"Records after popularity filter: {cleaned_df.count()}")

24/11/10 17:34:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

Records after popularity filter: 7966
